In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,50020
2,Huelva,Confirmados PDIA 14 días,2256
3,Huelva,Tasa PDIA 14 días,"439,6203986982871"
4,Huelva,Confirmados PDIA 7 días,1518
5,Huelva,Tasa PDIA 7 dias,"295,80840657092193"
6,Huelva,Total Confirmados,50230
7,Huelva,Curados,46455
8,Huelva,Fallecidos,424


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  50020.0


/tmp/ipykernel_79498/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  13993.0


/tmp/ipykernel_79498/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_79498/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_79498/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_79498/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 296 personas en los últimos 7 días 

Un positivo PCR cada 198 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,50020.0,1518.0,2256.0,513170.0,295.808407,439.620399,698.0
Huelva-Costa,29487.0,855.0,1251.0,289548.0,295.287828,432.052717,390.0
Huelva (capital),13993.0,486.0,726.0,143837.0,337.882464,504.738002,221.0
Condado-Campiña,15511.0,386.0,615.0,156231.0,247.070044,393.647868,172.0
Sierra de Huelva-Andévalo Central,4584.0,268.0,380.0,67391.0,397.679215,563.873514,129.0
Aracena,832.0,118.0,153.0,8255.0,1429.436705,1853.422168,49.0
Ayamonte,2200.0,65.0,71.0,21104.0,307.998484,336.429113,38.0
Bollullos Par del Condado,1484.0,71.0,106.0,14387.0,493.501077,736.776256,37.0
Aljaraque,1812.0,63.0,106.0,21474.0,293.378039,493.620192,31.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Puerto Moral,15.0,5.0,10.0,278.0,1798.561151,3597.122302,1.0
Cumbres Mayores,178.0,27.0,40.0,1749.0,1543.739280,2287.021155,9.0
Aracena,832.0,118.0,153.0,8255.0,1429.436705,1853.422168,49.0
Marines (Los),21.0,6.0,6.0,399.0,1503.759398,1503.759398,5.0
Villanueva de las Cruces,25.0,4.0,4.0,387.0,1033.591731,1033.591731,0.0
Fuenteheridos,21.0,5.0,6.0,709.0,705.218618,846.262341,2.0
Corteconcepción,51.0,1.0,4.0,536.0,186.567164,746.268657,0.0
Bollullos Par del Condado,1484.0,71.0,106.0,14387.0,493.501077,736.776256,37.0
Paterna del Campo,287.0,7.0,24.0,3457.0,202.487706,694.243564,3.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Corteconcepción,51.0,1.0,4.0,536.0,186.567164,746.268657,0.0,0.250000
Paterna del Campo,287.0,7.0,24.0,3457.0,202.487706,694.243564,3.0,0.291667
Zalamea la Real,131.0,1.0,3.0,3054.0,32.743942,98.231827,0.0,0.333333
Valverde del Camino,946.0,21.0,56.0,12750.0,164.705882,439.215686,9.0,0.375000
Palma del Condado (La),1519.0,19.0,41.0,10801.0,175.909638,379.594482,6.0,0.463415
Palos de la Frontera,1240.0,27.0,58.0,11742.0,229.943792,493.953330,9.0,0.465517
Cerro de Andévalo (El),202.0,1.0,2.0,2327.0,42.973786,85.947572,0.0,0.500000
Higuera de la Sierra,77.0,2.0,4.0,1291.0,154.918668,309.837335,2.0,0.500000
Puerto Moral,15.0,5.0,10.0,278.0,1798.561151,3597.122302,1.0,0.500000
